Passo 1 - Importar as bibliotecas

In [ ]:
%pip install yfinance==0.2.58
%pip install mplcyberpunk

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplcyberpunk

plt.style.use("cyberpunk")

Passo 2 - pegar os dados do Yahoo Finance

In [ ]:
ticker = "ALPA4.SA"
tabela_de_dados = yf.download(ticker, start= "2021-05-27", end= "2025-04-30", auto_adjust= False)



Passo 3 - Definir as janelas das médias móveis 

In [ ]:
janela_rapida = 7
janela_lenta = 40

Passo 4 - Calcular as médias móveis

In [ ]:
tabela_de_dados["Média Rápida"] = tabela_de_dados["Adj Close"].rolling(window=janela_rapida).mean()
tabela_de_dados["Média Lenta"] = tabela_de_dados["Adj Close"].rolling(window=janela_lenta).mean()



Passo 5 - Calcular o retorno diário da ação

In [ ]:
tabela_de_dados["retorno_diario"] = tabela_de_dados["Adj Close"].pct_change()
tabela_de_dados = tabela_de_dados.dropna()



Passo 6 - Criar os sinais de compra ou venda

In [ ]:
# se eu tiver comprado é 1
# se eu tiver neutro é 0
# se eu tiver vendido é -1

tabela_de_dados["comprado_ou_vendido"] = 0
tabela_de_dados["comprado_ou_vendido"] = np.where(tabela_de_dados["Média Rápida"] > tabela_de_dados["Média Lenta"], 1, -1)


tabela_de_dados["comprado_ou_vendido"] = tabela_de_dados["comprado_ou_vendido"].shift(1)



Passo 7 - Calcular o retorno do modelo

In [ ]:
tabela_de_dados["retorno_modelo"] = tabela_de_dados["retorno_diario"] * tabela_de_dados["comprado_ou_vendido"]
tabela_de_dados["retorno_acumulado_modelo"] = (1 + tabela_de_dados["retorno_modelo"]).cumprod() - 1



Passo 8 - Calcular o retorno da ação e do ibovespa no mesmo período

In [ ]:
tabela_de_dados["retorno_acum_acao"] = (1 + tabela_de_dados["retorno_diario"]).cumprod() - 1

ibov = yf.download("^BVSP", start=tabela_de_dados.index[0], end="2025-04-30", auto_adjust=False)["Adj Close"]

tabela_de_dados["retorno_ibov"] = ibov.pct_change().values

In [ ]:
tabela_de_dados

In [ ]:
tabela_de_dados["retorno_acum_ibov"] = (1 + tabela_de_dados["retorno_ibov"]).cumprod() - 1


Passo 9 - Criar o gráfico de retornos para observar o resultado da estratégia

In [ ]:
plt.plot(tabela_de_dados["retorno_acumulado_modelo"], label = "Estratégia")
plt.plot(tabela_de_dados["retorno_acum_acao"], label = "Ação")
plt.plot(tabela_de_dados["retorno_acum_ibov"], label = "ibovespa")
plt.legend()
plt.show()